<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    LOAD REAL TIME | LOAD DAY AHEAD
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Raw Data Downloading Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
        Each part of the following script was used to download the raw data for the Real Time and Day Ahead Load Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
from entsoe.exceptions import NoMatchingDataError, InvalidBusinessParameterError  # Import relevant exceptions
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil

 <div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. ENTSO-E RESTful API.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Connecting with the ENTSO-E API Tool.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
To donwload the neeeded data using the API tool, is mandatory to use a token autentication to connect and make the future request of data.
</div>

In [2]:
client = EntsoePandasClient(api_key='61e5bbbb-7e80-4540-a471-bd993873aa74')

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    3. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [3]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    4. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: Justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [4]:
# Year to which data refers to:
data_year = 2024
data_year_1 = data_year + 1

In [5]:
# Additional string to be appended
additional_path = "/RawData/Load_RealTime/"
additional_path_1 = "/RawData/Load_DayAhead/"
additional_path_2 = "/RawData/Load_RealTime/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
load_realtime_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_folder_path variable
load_dayAhead_folder_path = dispaSET_unleash_folder_path + additional_path_1

# Construct the Outage_Factors_Raw_Data_folder_path variable
load_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_2

print("load_realtime_folder_path:", load_realtime_folder_path)
print("load_dayAhead_folder_path:", load_dayAhead_folder_path)
print("load_raw_data_folder_path:", load_raw_data_folder_path)

load_realtime_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/
load_dayAhead_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/
load_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/


In [6]:
# Define other parameters
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    5. Country List Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Defining the list of countries according to the available data. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Just those countries that interchange flows with other countries different of the ones modelled in Dispa-SET are defined in the list.
</div>

In [7]:
# Define a list of country codes
load_data_country_list = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "GR", "ES", "FI", "FR", "HR", "HU", 
                          "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"]

In [8]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(load_raw_data_folder_path, file_name)

# Ensure the directory exists
os.makedirs(load_raw_data_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(load_data_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
load_data_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv'


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [9]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"load_realtime_folder_path:                                 {load_realtime_folder_path}")   
print (f"load_dayAhead_folder_path:                                 {load_dayAhead_folder_path}")
print (f"load_raw_data_folder_path:                                 {load_realtime_folder_path}")
print (f"load_data_country_list_file:                               {load_data_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2024
load_realtime_folder_path:                                 /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/
load_dayAhead_folder_path:                                 /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/
load_raw_data_folder_path:                                 /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/
load_data_country_list_file:                               /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman; color:skyblue">
Defining the sub-folders where all the load raw data is saved. 
</div>

In [10]:
# Convert data_year to string if it's not already
data_year = str(data_year)

# Read the CSV file into a DataFrame
df = pd.read_csv(load_data_country_list_file)

# Ensure the column 'Country_From' exists
if 'Country_From' not in df.columns:
    raise ValueError("Column 'Country_From' does not exist in the CSV file")

# Define the base directory where subfolders will be created
base_directory = os.path.join(load_raw_data_folder_path, data_year)

# Create a list to hold the paths of the created subfolders
country_folder_paths = []

# Create subfolders and save their paths
for country in df['Country_From']:
    # Create the subfolder path
    subfolder_path = os.path.join(base_directory, country)
    
    # Create the subfolder if it doesn't exist
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Append the subfolder path to the list
    country_folder_paths.append(subfolder_path)

# Add the new column 'Country_Folder' to the DataFrame
df['Country_Folder'] = country_folder_paths

# Save the updated DataFrame back to the CSV file
df.to_csv(load_data_country_list_file, index=False)

print(f"Updated CSV file saved to '{load_data_country_list_file}' with new subfolders created.")

Updated CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv' with new subfolders created.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    6. Raw Data Download
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Donwloading the real time and day ahead load raw data. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The load data is downloaded in separate files for each country.
<br>
Since the Acronym of Grece in the downloaded data is 'GR' and the Dispa-SET format for the country is 'EL'. All the needed changes in the used variables are done.
<br>

In [11]:
# Define the start and end timestamps
start = pd.Timestamp(f'{data_year}0101', tz='Europe/Brussels')
end = pd.Timestamp(f'{data_year_1}0101', tz='Europe/Brussels')

# Read the CSV file into a DataFrame
df = pd.read_csv(load_data_country_list_file)

# Ensure the necessary columns exist
required_columns = ['Country_From', 'Country_Folder']
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' does not exist in the CSV file")

# Add a new column for the raw data file paths
df['Raw_Data_File_Path'] = ""

# Lists to keep track of countries with connection errors and other errors
retry_list = []
error_list = []

# Function to check if data is valid
def is_valid_data(data):
    return data is not None and not data.empty

# Function to process each country
def process_country(country_code, country_folder):
    try:
        # Query load and forecast data
        load_forecast_data = client.query_load_and_forecast(country_code, start=start, end=end)
        
        # Check if the data is valid
        if is_valid_data(load_forecast_data):
            # Convert the index to a column
            load_forecast_data = load_forecast_data.reset_index()
            
            # Define the output file path
            output_file = os.path.join(country_folder, f'{data_year}_1.csv')
            
            # Save the DataFrame to a CSV file, including the index as a column
            load_forecast_data.to_csv(output_file, index=False)

            print(f"Data for {country_code} saved to '{output_file}'")
            return output_file  # Return the path of the file
        else:
            print(f"No valid data for {country_code}. Skipping.")
            return None
    except ConnectionError:
        print(f"Connection error for {country_code}. Adding to retry list.")
        retry_list.append((country_code, country_folder))
        return None
    except (NoMatchingDataError, InvalidBusinessParameterError) as e:
        print(f"No matching data or invalid parameters for {country_code}. Error: {str(e)}")
        error_list.append((country_code, country_folder))
        return None
    except Exception as e:
        print(f"An unexpected error occurred for {country_code}. Error: {str(e)}")
        error_list.append((country_code, country_folder))
        return None

# Process each country
first_successful_file = None
for index, row in df.iterrows():
    country_code = row['Country_From']
    country_folder = row['Country_Folder']
    
    # Create the directory if it does not exist
    os.makedirs(country_folder, exist_ok=True)
    
    # Process the country and get the path of the output file
    output_file = process_country(country_code, country_folder)
    
    if output_file:
        # Update the DataFrame with the path to the downloaded file
        df.at[index, 'Raw_Data_File_Path'] = output_file
        
        # Track the first successful file for error handling
        if not first_successful_file:
            first_successful_file = output_file

# Retry process for countries with connection errors
if retry_list:
    print("Retrying for countries with connection errors...")
    for country_code, country_folder in retry_list:
        output_file = process_country(country_code, country_folder)
        if output_file:
            # Update the DataFrame with the path to the downloaded file
            df.loc[df['Country_From'] == country_code, 'Raw_Data_File_Path'] = output_file

# Handle countries with other errors by copying data from the first successful file
if first_successful_file:
    print(f"Using {first_successful_file} for handling errors.")
    successful_data = pd.read_csv(first_successful_file)
    first_column = successful_data.iloc[:, 0]  # First column data
    second_and_third_headers = successful_data.columns[1:3].tolist()  # Second and third column headers
    
    for country_code, country_folder in error_list:
        error_output_file = os.path.join(country_folder, f'{data_year}_1.csv')
        # Create a DataFrame with the first column
        error_data = pd.DataFrame({successful_data.columns[0]: first_column})
        # Add dummy columns for the second and third headers
        for header in second_and_third_headers:
            error_data[header] = pd.Series(dtype='float64')
        # Save the DataFrame to a CSV file
        error_data.to_csv(error_output_file, index=False)
        print(f"Error file created for {country_code} at '{error_output_file}'")
        # Update the DataFrame with the path to the error file
        df.loc[df['Country_From'] == country_code, 'Raw_Data_File_Path'] = error_output_file

# Save the updated DataFrame with the file paths
df.to_csv(load_data_country_list_file, index=False)

print("All data processing completed.")

Data for AT saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/AT/2024_1.csv'
Data for BE saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/BE/2024_1.csv'
Data for BG saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/BG/2024_1.csv'
Data for CH saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/CH/2024_1.csv'
Data for CY saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/CY/2024_1.csv'
Data for CZ saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/CZ/2024_1.csv'
Data for DE saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/DE/2024_1.csv'
Data for DK saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/DK/2024_1.csv'
Data for EE saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2024/EE/2024_1.csv'
Data for GR saved t

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    6.1 Downloading Errors
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
In the downloading process, there are some countries which related data cannot be obtained using the API connection, i.e.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Cyprus (CY).
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
The data of these countries are got manually from the following link.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
<a href="https://transparency.entsoe.eu/load-domain/r2/totalLoadR2/show" style="color:skyblue;">https://transparency.entsoe.eu/load-domain/r2/totalLoadR2/show</a></div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    7. Cleaning Folder
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
All the raw data files are storage in the following local directory inside a sub-folder named as the value defined by 'data_year' variable. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
/local/directory/to/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources
</div>


In [12]:
def erase_csv_file(file_path):
  """Erases a CSV file.

  Args:
    file_path: The path to the CSV file.
  """

  if os.path.exists(file_path):
    os.remove(file_path)
    print(f"File erased: {file_path}")
  else:
    print(f"File does not exist: {file_path}")

# Example usage:
file_path = load_data_country_list_file
erase_csv_file(file_path)

File erased: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv
